In [2]:
#finallll

import tkinter as tk
from tkinter import messagebox, simpledialog, ttk
import random
import time
import json
from PIL import Image, ImageTk, ImageDraw

class ParkingSpace:
    def __init__(self, id):
        self.id = id
        self.is_occupied = False

class ParkingLot:
    def __init__(self, num_spaces):
        self.spaces = [ParkingSpace(i) for i in range(1, num_spaces + 1)]

    def get_available_spaces(self):
        return [space for space in self.spaces if not space.is_occupied]

class Car:
    def __init__(self, id, vehicle_type):
        self.id = id
        self.vehicle_type = vehicle_type
        self.entry_time = 0
        self.exit_time = 0

class PricingModel:
    def __init__(self, base_rate_per_minute):
        self.base_rate_per_minute = base_rate_per_minute

    def calculate_cost(self, duration):
        return duration * self.base_rate_per_minute / 60

class SmartParkingSystem:
    def __init__(self, parking_lot, pricing_model):
        self.parking_lot = parking_lot
        self.pricing_model = pricing_model
        self.parked_cars = {}

    def park_car(self, car):
        available_spaces = self.parking_lot.get_available_spaces()

        if not available_spaces:
            messagebox.showinfo("Parking Full", "Parking lot is full. Unable to park the car.")
            return

        selected_space = random.choice(available_spaces)
        selected_space.is_occupied = True

        car.entry_time = time.time()
        self.parked_cars[car.id] = {"space_id": selected_space.id, "entry_time": car.entry_time, "exit_time": 0, "vehicle_type": car.vehicle_type}

        self.show_receipt(car.id, selected_space.id, car.vehicle_type)

    def unpark_car(self, car_id):
        if car_id not in self.parked_cars:
            messagebox.showinfo("Car Not Found", f"Car {car_id} is not currently parked.")
            return

        space_id = self.parked_cars[car_id]["space_id"]
        entry_time = self.parked_cars[car_id]["entry_time"]

        space = next((s for s in self.parking_lot.spaces if s.id == space_id), None)

        if space and space.is_occupied:
            space.is_occupied = False

            exit_time = time.time()
            parking_duration = exit_time - entry_time
            parking_cost = self.pricing_model.calculate_cost(parking_duration)

            self.parked_cars[car_id]["exit_time"] = exit_time

            self.show_receipt(car_id, space_id, parking_duration, parking_cost, self.parked_cars[car_id]["vehicle_type"])
        else:
            messagebox.showinfo("No Car Found", f"No car found in space {space_id}")


    def show_receipt(self, car_id, space_id, parking_duration=None, parking_cost=None, vehicle_type=None):
        entry_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(self.parked_cars[car_id]["entry_time"]))
        exit_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(self.parked_cars[car_id]["exit_time"]))

        receipt = f"Receipt for Car {car_id}\n"\
                  f"Vehicle Type: {vehicle_type}\n"\
                  f"Parking Space: {space_id}\n"\
                  f"Entry Time: {entry_time}\n"

        if parking_duration is not None and parking_cost is not None:
            receipt += f"Parking Duration: {parking_duration:.2f} seconds\n"\
                       f"Exit Time: {exit_time}\n"\
                       f"Cost: ₹{parking_cost:.2f}"
        else:
            receipt += "Exit Time: Not yet exited"

        messagebox.showinfo("Receipt", receipt)

    def format_parking_history(self):
        formatted_history = {}
        for car_id, info in self.parked_cars.items():
            entry_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(info["entry_time"]))
            exit_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(info["exit_time"])) if info["exit_time"] else "Not yet exited"
            formatted_history[car_id] = {
                "vehicle_type": info["vehicle_type"],
                "space_id": info["space_id"],
                "entry_time": entry_time,
                "exit_time": exit_time
            }
        return formatted_history

    def get_parking_history(self):
        return self.parked_cars

    def get_occupied_spaces_grid(self):
        return [space.is_occupied for space in self.parking_lot.spaces]

class WelcomePage:
    def __init__(self, master):
        self.master = master
        self.master.title("Welcome to Smart Car Parking System")

        self.welcome_label = tk.Label(master, text="Hi! I am to Park AI. Welcome to Smart Car Parking System!", font=("Helvetica", 16))
        self.welcome_label.pack(pady=20)

        self.continue_button = tk.Button(master, text="Continue", command=self.show_parking_system)
        self.continue_button.pack()

    def show_parking_system(self):
        self.master.destroy()  # Close the welcome page
        num_spaces = 10
        base_rate_per_minute = 0.1  # Example rate in Rupees per minute

        parking_lot = ParkingLot(num_spaces)
        pricing_model = PricingModel(base_rate_per_minute)
        parking_system = SmartParkingSystem(parking_lot, pricing_model)

        root = tk.Tk()
        app = ParkingGUI(root, parking_system)
        root.mainloop()

class ParkingGUI:
    def __init__(self, master, parking_system):
        self.master = master
        self.master.title("Smart Car Parking System.")

        self.parking_system = parking_system

        self.welcome_label = tk.Label(master, text="Kindly fill in the details of your vehicle.", font=("Helvetica", 16))
        self.welcome_label.pack(pady=10)

        self.label = tk.Label(master, text="Enter Car ID:")
        self.label.pack()

        self.car_id_entry = tk.Entry(master)
        self.car_id_entry.pack()

        self.vehicle_type_label = tk.Label(master, text="Select Your Vehicle Type:")
        self.vehicle_type_label.pack()

        vehicle_types = ["2 Wheeler", "4 Wheeler", "Bus", "Truck"]
        self.vehicle_type_var = tk.StringVar(master)
        self.vehicle_type_var.set(vehicle_types[0])  # Set the default value

        vehicle_type_menu = ttk.Combobox(master, textvariable=self.vehicle_type_var, values=vehicle_types)
        vehicle_type_menu.pack(pady=10)

        self.park_button = tk.Button(master, text="Park Car", command=self.park_car)
        self.park_button.pack()

        self.unpark_button = tk.Button(master, text="Unpark Car", command=self.unpark_car)
        self.unpark_button.pack()

        self.history_button = tk.Button(master, text="Parking History", command=self.show_parking_history)
        self.history_button.pack()

        self.grid_label = tk.Label(master, text="Parking Spaces:")
        self.grid_label.pack()

        self.grid_frame = tk.Frame(master)
        self.grid_frame.pack()

        self.update_grid_view()

    def park_car(self):
        car_id = self.car_id_entry.get()
        vehicle_type = self.vehicle_type_var.get()

        if car_id.isdigit():
            car_id = int(car_id)
            car = Car(car_id, vehicle_type)
            self.parking_system.park_car(car)
            self.update_grid_view()
        else:
            messagebox.showinfo("Invalid Input", "Please enter a valid car ID.")

    def unpark_car(self):
        car_id = self.car_id_entry.get()
        if car_id.isdigit():
            car_id = int(car_id)
            self.parking_system.unpark_car(car_id)
            self.update_grid_view()

            # After unparking, show payment options
            self.show_payment_options()
        else:
            messagebox.showinfo("Invalid Input", "Please enter a valid car ID.")

    def show_payment_options(self):
        payment_options = ["Cash", "Card", "UPI"]

        payment_var = tk.StringVar(self.master)
        payment_var.set(payment_options[0])  # Set the default value

        payment_label = tk.Label(self.master, text="Select payment option:")
        payment_label.pack(pady=5)

        payment_menu = ttk.Combobox(self.master, textvariable=payment_var, values=payment_options)
        payment_menu.pack(pady=10)

        payment_button = tk.Button(self.master, text="Proceed", command=lambda: self.handle_payment(payment_var.get()))
        payment_button.pack(pady=10)

    def handle_payment(self, selected_option):
        if selected_option == "Cash":
            messagebox.showinfo("Payment", "Please pay at the counter.")
            # After payment, show the thank you message
            self.show_thank_you_message()
        elif selected_option == "Card":
            self.show_card_swipe_prompt()
        elif selected_option == "UPI":
            self.show_qr_code_prompt()

    def show_card_swipe_prompt(self):
        messagebox.showinfo("Payment", "Please swipe your card.")

    def show_qr_code_prompt(self):
        # For demonstration purposes, generate a random QR code image
        qr_code_image = Image.new('RGB', (200, 200), color='white')
        qr_code_draw = ImageDraw.Draw(qr_code_image)
        qr_code_draw.text((10, 10), "Random UPI QR Code", fill='black')

        # Display the image
        img = ImageTk.PhotoImage(qr_code_image)
        panel = tk.Label(self.master, image=img)
        panel.image = img
        panel.pack()
        messagebox.showinfo("Payment", "Please scan the QR code.")
        # After payment, show the thank you message
        self.show_thank_you_message()

    def show_thank_you_message(self):
        messagebox.showinfo("Thank You", "Thank you for using our Smart Parking System! Visit Again.")

    def show_parking_history(self):
        history = self.parking_system.format_parking_history()
        if history:
            history_str = json.dumps(history, indent=2)
            messagebox.showinfo("Parking History", f"Parking History:\n{history_str}")
        else:
            messagebox.showinfo("No History", "No parking history available.")

    def update_grid_view(self):
        occupied_spaces = self.parking_system.get_occupied_spaces_grid()

        for widget in self.grid_frame.winfo_children():
            widget.destroy()

        for i, is_occupied in enumerate(occupied_spaces, start=1):
            label_text = f"Space {i}: {'Occupied' if is_occupied else 'Available'}"
            label = tk.Label(self.grid_frame, text=label_text, width=20, height=2)

            if is_occupied:
                label.configure(bg="red", fg="white")
            else:
                label.configure(bg="green", fg="white")

            label.pack()

if __name__ == "__main__":
    welcome_root = tk.Tk()
    welcome_page = WelcomePage(welcome_root)
    welcome_root.mainloop()
